<a href="https://colab.research.google.com/github/elsa9421/Interactive-IPython-Demos/blob/main/Kernel_Nearest_centroid_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates the *Nearest Centroid Classifier* using the following kernels-
* linear
* quadratic
* polynomial
* cubic polynomial
* gaussian

In [ ]:
from sklearn.datasets import make_moons,make_blobs,make_circles
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import numpy as np

from ipywidgets import interact,interactive,interactive_output, fixed, interact_manual
import ipywidgets as widgets
import datetime

import time


order=2
sigma=.1
C_param=1   
def Linear_kernel(X1,X2):
  '''
  For a linear kernel K(xi,xj)=⟨xi,xj⟩ i.e dot(X,X.T)
 
  Input:
  -X1 , of shape(N1,2)
  -X2 , of shapw(N2,2)
  

  Output:
  -return dot(X1,X2.T), of shape(N1,N2)


  '''
  #return np.dot(X1,X2.T)  # (N,N)
  X1=X1.reshape(-1,2)
  X2=X2.reshape(-1,2)
  return np.dot(X1,X2.T)
#  return np.sum(np.multiply(X1.reshape(-1,2),X2.reshape(-1,2)),axis=1)

def polynomial_kernel(X1,X2):
  '''
  For degree-d polynomials, the polynomial kernel is defined as

  K(x,x')=(x.T*x'+c)^d
  here, c ≥ 0 is a free parameter trading off the influence of higher-order 
  versus lower-order terms in the polynomial. When c = 0, the kernel is called homogeneous

  '''
  global order
  global C_param
  c=C_param
  
  X1=X1.reshape(-1,2)
  X2=X2.reshape(-1,2)
  #return np.power(np.sum(np.multiply(X1.reshape(-1,2),X2.reshape(-1,2)),axis=1),order)

  return np.power((c+np.dot(X1,X2.T)),order)

def Gaussian_kernel(X1,X2):
  '''
  K(x ,x')=exp(-||x - x'||^2/(2*sigma^2))
  '''
  global sigma


  X1=X1.reshape(-1,2)
  X2=X2.reshape(-1,2)
  X1_new=np.repeat(X1[np.newaxis,:, :], X2.shape[0], axis=0)  #(100,000,N,2)
  X2_new=X2[:,:,np.newaxis].transpose(0,2,1)


  out=np.exp(-np.divide(np.power(np.linalg.norm(X1_new-X2_new,axis=2),2),(2*sigma**2)))


  
  return out




def predict(Testpoints,kernel,X_class0,X_class1,n_class0,n_class1,degree):
  K=kernel
  k_x=[]

  if K==Gaussian_kernel:
    bias=0
    term2=(1/n_class0)*np.sum(K(X_class0,Testpoints),axis=1)
    term1=(1/n_class1)*np.sum(K(X_class1,Testpoints),axis=1)

  else:
    #bias=0
    bias=((1/(n_class0**2))*np.sum(K(X_class0,X_class0))-(1/(n_class1**2))*np.sum(K(X_class1,X_class1)))*0.5
    term2=(1/n_class0)*np.sum(K(Testpoints,X_class0),axis=1)
    term1=(1/n_class1)*np.sum(K(Testpoints,X_class1),axis=1)

  f_x=term1-term2+bias
  Z=np.zeros(f_x.shape)
  Z[f_x>0]=1
  Z[f_x<0]=-1

  return Z


 
  
  


#def KNCC(kernel,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,degree=2,sig=1):
def KNCC(kernel,data_type='Linear',degree=2,sig=1,c=1):
  '''
  Plot kernel- Nearest Centroid Classifier
  '''
  start_time=time.time()
  if data_type=='Non-Linear Moons':
    X_train, y_train = make_moons(n_samples=100, noise=0.1,random_state=14)
  elif data_type=='Non-Linear':
    X_train, y_train = make_blobs(n_samples=200, centers=[(-1, 1), (0.5,0.5)], random_state=14, cluster_std=0.65)
  elif data_type=='Circle':
    X_train, y_train = make_circles(n_samples=100,noise=0.05,random_state=14,factor=0.2)
    
   # X_train[:,1]=X_train[:,1]+2

  y_train[y_train==0]=-1

  n_class0=len(y_train[y_train==-1])
  n_class1=len(y_train[y_train==1])
  
  
  X_train_class0=X_train[y_train==-1]
  
  X_train_class1=X_train[y_train==1]
  global order
  global sigma
  global C_param
  
  h=0.01
  if kernel=="Linear":
    K=Linear_kernel
  
  if kernel=="Quadratic":
    K=polynomial_kernel
    order=2
    C_param=c
  
  if kernel=="Polynomial":
    K=polynomial_kernel
    order=degree
    print("Degree=",order)
    C_param=c

  if kernel=="Cubic":
    K=polynomial_kernel
    order=3
    C_param=c

  if kernel=="Gaussian":
    K=Gaussian_kernel
    sigma=sig
    h=0.02
 

  plt.figure(figsize=(7,7))
  plt.scatter(X_train[y_train==-1,0], X_train[y_train==-1,1], c='b', marker="o", label='Class -1')
  plt.scatter(X_train[y_train==1,0], X_train[y_train==1,1], c='g', marker="x", label='Class 1')


 
  x_min, x_max = X_train[:, 0].min()-.1, X_train[:, 0].max()+.1#%time = 1.23s
  y_min, y_max = X_train[:, 1].min() -.1, X_train[:, 1].max() +.1

  xx,yy=np.meshgrid(np.arange(x_min, x_max,h), np.arange(y_min, y_max,h))
  #print("XX=",xx.ravel().shape)



  Z = predict(np.c_[xx.ravel(), yy.ravel()],kernel=K,X_class0=X_train_class0,X_class1=X_train_class1,n_class0=n_class0,n_class1=n_class1,degree=order)
  Z = Z.reshape(xx.shape)
  #plt.contour(xx, yy, Z, alpha=0.8,levels=[0],colors='black')
  plt.contourf(xx, yy, Z, alpha=0.3,levels=[-1,0,1],colors=['b','g'])
  plt.grid()
  
  print ("Time taken=",time.time()-start_time)
  print("sigma=",sigma)


    
 

# %time KNCC(kernel="Linear")
# %time KNCC(kernel="Quadratic")
# %time KNCC(kernel="Cubic")
# %time KNCC(kernel="Polynomial",degree=4)
# %time KNCC(kernel="Gaussian")







kernel_widget=widgets.Dropdown(
                      options=['Linear', 'Quadratic', 'Cubic','Polynomial','Gaussian'],
                      value='Linear',
                      description='Kernel:',
                      disabled=False,
                      )

data_widget=widgets.Dropdown(
                      options=['Non-Linear', 'Circle','Non-Linear Moons'],
                      value='Non-Linear',
                      description='Data_type:',
                      disabled=False,
                      )

degree_widget=widgets.IntText(value=2,
                                 min=1,
                                 max=63,
                                 step=1,
                                 description='Degree',
                                 continuous_update=False)

# c_widget=widgets.FloatSlider(value=1,
#                                  min=-300,
#                                  max=500,
#                                  step=.1,
#                                  description='c:',
#                                  continuous_update=False)

poly_widget=widgets.HBox([degree_widget])



sigma_slider=widgets.FloatSlider(value=0.1,
                                 min=0.001,
                                 max=2.25,
                                 step=.04,
                                 description='sigma',
                                 continuous_update=False)



sigma_text=widgets.FloatText(value=0.1,
                                 min=0.001,
                                 max=2.25,
                                 step=.04,
                                 description='sigma',
                                 continuous_update=False)






widgets.link((sigma_slider, 'value'), (sigma_text, 'value'))
sigma_widget=widgets.HBox([sigma_slider,sigma_text])



out=interactive_output(KNCC,{"kernel":kernel_widget,"data_type":data_widget,"degree":degree_widget,"sig":sigma_text})


display(kernel_widget,data_widget,poly_widget,sigma_widget,out)








Dropdown(description='Kernel:', options=('Linear', 'Quadratic', 'Cubic', 'Polynomial', 'Gaussian'), value='Lin…

Dropdown(description='Data_type:', options=('Non-Linear', 'Circle', 'Non-Linear Moons'), value='Non-Linear')

Output()